In [85]:
## WIZDRIVE
## Code for running Wizardry-esque early dungeon crawl games
## by Nicholas Smith
## Copyright (c) 2025

In [86]:
## import statements go here
import io
import os
from sys import exit
import pygame
from time import sleep

In [87]:
## Global variables

currentMap = None                                       # Current level map
mapWidth = 0                                            # Current map's width
mapHeight = 0                                           # Current map's height
player = None                                           # Player object
playerFacing = 1                                        # Direction player is facing (1 for North, 2 for East, 3 for South, 4 for West)

savefile = None                                         # Current loaded save file

enemies = []                                            # List of all enemy Sprites
items = []                                              # List of all item Sprite
VALID_FAMILIES = ['player','enemy','item']              # list of all Sprite families in game
VALID_TYPES = ['player','enemy','item']                 # List of all Sprite types

screen = None                                           # pygame display surface
pointsOne = None                                        # Points denoting beginning of relative space (1, 0)
pointsTwo = None                                        # Points denoting beginning of relative space (2, 0)
pointsThree = None                                      # Points denoting beginning of relative space (3, 0)
pointsFour = None                                       # Points denoting beginning of relative space (4, 0)
sidePointsOneOne = None                                 # Points for connecting walls of spaces (1, -1) and (1, 1) forward
sidePointsTwoOne = None                                 # Points for connecting walls of spaces (2, -1) and (2, 1) forward
sidePointsThreeOne = None                               # Points for connecting walls of spaces (3, -1) and (3, 1) forward
sidePointsThreeTwo = None                               # Points for connecting walls of spaces (3, -2) and (3, 2) forward
screenWidth = 0                                         # Width of pygame window in pixels
screenHeight = 0                                        # Height of pygame window in pixels

### Global constants

WALL    = 0b0
ROOM    = 0b1
VISITED = 0b10
NONBLOCKER = 0b100

In [88]:
## Player class

class Player():
    __location = (0,0)
    __facing = 1

    def __init__(self, coordinates, direction):
        self.__location = coordinates
        self.__facing = direction
        print(self.__location)
        print(self.__facing)

    def getLocation(self):
        return self.__location
    
    def getFacing(self):
        return self.__facing

    def turn(self, direction):
        match direction:
            case "left":
                self.__facing -= 1
                if self.__facing == 0:
                    self.__facing = 4
            case "right":
                self.__facing += 1
                if self.__facing == 5:
                    self.__facing = 1
            case "around":
                self.turn("right")
                drawView(getVisible())
                sleep(0.25)
                self.turn("right")
            case _:
                raise ValueError
        print(self.__facing)
        return

            
    def moveForward(self):
        global currentMap
        newLocation = 0
        match self.__facing:
            case 1:
                newLocation = (self.__location[0] - 1, self.__location[1])
            case 2:
                newLocation = (self.__location[0], self.__location[1] + 1)
            case 3:
                newLocation = (self.__location[0] + 1, self.__location[1])
            case 4:
                newLocation = (self.__location[0], self.__location[1] - 1)
            case _:
                raise ValueError
        try:
            if (currentMap[newLocation[0]][newLocation[1]] & ROOM):
                self.__location = newLocation
                currentMap[newLocation[0]][newLocation[1]] = (currentMap[newLocation[0]][newLocation[1]] | VISITED)
            else:
                print("OOF!")
        except IndexError:
            print("OOF")
        return
    

In [89]:
## Item class

class Item(pygame.sprite.Sprite):
    __location = (0,0)
    __facing = 0
    __blocking = False
    __type = ""
    
    def __init__(self, location, facing, blocking, type):
        self.__location = location
        self.__facing = facing
        self.__blocking = blocking
        self.__type = type
        return self

In [90]:
########################## MAP CODE #################################
## Load current map into memory from disk
## Returns a Map object

def loadMap(mapfile):

    global mapWidth
    global mapHeight
    global player

    mapstream = open(mapfile, "r", encoding="utf-8")
    try:
        # Ensure the filetype is correct
        teststring = mapstream.readline()
        if teststring != "Wizdrive map\n":
            raise ValueError("Loaded file was not a valid map.")
        
        ## get map info for player and initialize player object
        y = int(mapstream.readline())
        x = int(mapstream.readline())
        d = int(mapstream.readline())
        player = Player((y,x), d)

        # Split map into a two-dimensional array
        map = mapstream.read()
        map = map.split("\n")
        map[len(map)-1].strip()
        for l, line in enumerate(map):
            map[l] = line.split(",")
    
    # Set mapHeight and mapWidth based on this new array
        mapHeight = len(map)
        mapWidth = len(map[0])
    # Sanity check; all rows must be the length of mapWidth and there must be mapHeight rows
        for r in range(mapHeight):
            for c in range(mapWidth):
                testvar = map[r][c]
                map[r][c] = int(map[r][c])
    except Exception as e:
        print(e)
        exit(1)
    for line in map:
        print(line)
    return map

## Code to get a map of visible spaces from the current map and facing. Values are a 4x3 matrix of the spaces immediately
## in front of the player, with the "front" determined by the value of global varible playerfacing.

def getVisible():
    playerFacing = player.getFacing()
    playerLocation = player.getLocation()
    visible = [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
    print(playerLocation)
    # Player facing North
    match playerFacing:
        case 1:
            for y in range(4):
                r = y + (playerLocation[0] - 3)
                if (r >= len(currentMap)) or (r < 0):
                    continue
                for x in range(3):
                    c = x + (playerLocation[1] - 1)
                    if (c >= len(currentMap[1])) or (c < 0):
                        continue
                    visible[y][x] = currentMap[r][c]
        # Player facing East
        case 2:
            for y in range(4):
                c = playerLocation[1] + 3 - y
                if c >= len(currentMap[0]) or c < 0:
                    continue
                for x in range(3):
                    r = x + playerLocation[0] - 1
                    if r >= len(currentMap) or r < 0:
                        continue
                    visible[y][x] = currentMap[r][c]
        # Player is facing South
        case 3:
            for y in range(4):
                r = playerLocation[0] + 3 - y
                if (r >= len(currentMap)) or (r < 0):
                    continue
                for x in range(3):
                    c = playerLocation[1] + 1 - x
                    if (c >= len(currentMap[0])) or (c < 0):
                        continue
                    visible[y][x] = currentMap[r][c]
        # Player is facing West
        case 4:
            for y in range(4):
                c = y + playerLocation[1] - 3
                if (c >= len(currentMap[0])) or (c < 0):
                    continue
                for x in range(3):
                    r = playerLocation[0] + 1 - x
                    if (r >= len(currentMap)) or (r < 0):
                        continue
                    visible[y][x] = currentMap[r][c]
        case _:
            raise ValueError
    print(visible[0])
    print(visible[1])
    print(visible[2])
    print(visible[3])

    return visible

In [91]:
## Main game loop

def gameLoop():
    run = True
    while run:
        # Poll continuously for keyboard input
        # When one is found, parse it and perform the action
        # for gameEvent in pygame.event.get():
        for gameEvent in pygame.event.get():
            
            # If a key was pressed, toss it to the keyboardHandler() method
            if gameEvent.type == pygame.KEYDOWN:
                keyboardHandler(gameEvent.key)
            
            # If a quit command was sent, end the loop and exit the program
            elif gameEvent.type == pygame.QUIT:
                run = False
                pygame.quit()
    exit(0)
    # Enemies take turns
    # Repeat
    return

In [92]:
##################################### USER CONTROL CODE #######################################



In [93]:
## Handle keyboard input

def keyboardHandler(pressed):
    global playerFacing
    match pressed:

        case pygame.K_UP:
            player.moveForward()
            drawView(getVisible())
            print("K_UP pressed")

        # Down arrow turns the player around
        case pygame.K_DOWN:
            player.turn("around")
            drawView(getVisible())
            print("K_DOWN pressed")

        # Left arrow turns the player left
        case pygame.K_LEFT:
            player.turn("left")
            drawView(getVisible())
            print("K_LEFT pressed")

        # Right arrow turns the player right
        case pygame.K_RIGHT:
            player.turn("right")
            drawView(getVisible())
            print("K_RIGHT pressed")

        case _:
            print("Error?")
    return

In [ ]:
###################################### GRAPHICS CODE #######################################


## Initialize pygame and set graphics constants based on size of window

def initialize():
    # Method uses all global graphics variables
    global screen
    global pointsOne
    global pointsTwo
    global pointsThree
    global pointsfour
    global sidePointsOneOne
    global sidePointsTwoOne
    global sidePointsThreeOne
    global sidePointsThreeTwo
    global screenWidth
    global screenHeight

    # initialize Pygame
    pygame.init()
    (screenWidth, screenHeight) = (800, 800)
    pointsOne = ((screenWidth*.2, screenHeight*.2), (screenWidth*.2, screenHeight*.8), (screenWidth*.8, screenHeight*.2),(screenWidth*.8, screenHeight*.8))
    pointsTwo = ((screenWidth*.35, screenHeight*.35), (screenWidth*.35, screenHeight*.65), (screenWidth*.65, screenHeight*.35),(screenWidth*.65, screenHeight*.65))
    pointsThree = ((screenWidth*.45, screenHeight*.45), (screenWidth*.45, screenHeight*.55), (screenWidth*.55, screenHeight*.45),(screenWidth*.55, screenHeight*.55))
    pointsfour = ((screenWidth*.475, screenHeight*.475), (screenWidth*.475, screenHeight*.525), (screenWidth*.525, screenHeight*.475), (screenWidth*.525, screenHeight*.525))
    sidePointsOneOne = ((0,0), (0,0), (0,0), (0,0))
    sidePointsTwoOne = ((0,0), (0,0), (0,0), (0,0))
    sidePointsThreeOne = ((0,0), (0,0), (0,0), (0,0))
    sidePointsThreeTwo = ((0,0), (0,0), (0,0), (0,0))

    screen = pygame.display.set_mode(size=(screenWidth, screenHeight))
    screen.fill("black")

## Draw cell for currently occupied space

def __drawZero():
    pygame.draw.line(screen, "white", (0, 0), pointsOne[0])
    pygame.draw.line(screen, "white", (0, screenHeight), pointsOne[1])
    pygame.draw.line(screen, "white", (screenWidth, 0), pointsOne[2])
    pygame.draw.line(screen, "white", (screenWidth, screenHeight), pointsOne[3])
    pygame.draw.line(screen, "white", pointsOne[0], pointsOne[2])
    pygame.draw.line(screen, "white", pointsOne[0], pointsOne[1])
    pygame.draw.line(screen, "white", pointsOne[2], pointsOne[3])
    pygame.draw.line(screen, "white", pointsOne[1], pointsOne[3])
    # pygame.draw.line(screen, "white", (screenWidth*0.5, screenHeight*0.5), (screenWidth*(-0.2), screenHeight*(0.2)))
    return

## Draw cell to left of player
def __drawZeroLeft():
    pygame.draw.line(screen, "white", pointsOne[0], (0, pointsOne[0][1]))
    pygame.draw.line(screen, "white", pointsOne[1], (0, pointsOne[1][1]))
    return

## Draw cell to right of player
def __drawZeroRight():
    pygame.draw.line(screen, "white", pointsOne[2], (screenWidth, pointsOne[2][1]))
    pygame.draw.line(screen, "white", pointsOne[3], (screenWidth, pointsOne[3][1]))
    return

## Draw cell one space ahead of player
def __drawOne():
    for i in range(4):
        pygame.draw.line(screen, "white", pointsOne[i], pointsTwo[i])
    pygame.draw.line(screen, "white", pointsTwo[0], pointsTwo[2])
    pygame.draw.line(screen, "white", pointsTwo[0], pointsTwo[1])
    pygame.draw.line(screen, "white", pointsTwo[2], pointsTwo[3])
    pygame.draw.line(screen, "white", pointsTwo[1], pointsTwo[3])
    return

## Draw cell one left
def __drawOneLeft():
    pygame.draw.line(screen, "white", pointsTwo[0], (pointsOne[0][0], pointsTwo[0][1]))
    pygame.draw.line(screen, "white", pointsTwo[1], (pointsOne[1][0], pointsTwo[1][1]))
    return

## Draw cell one right
def __drawOneRight():
    pygame.draw.line(screen, "white", pointsTwo[2], (pointsOne[2][0], pointsTwo[2][1]))
    pygame.draw.line(screen, "white", pointsTwo[3], (pointsOne[3][0], pointsTwo[3][1]))
    return

## Draw cell two spaces ahead of player
def __drawTwo():
    for i in range(4):
        pygame.draw.line(screen, "white", pointsTwo[i], pointsThree[i])
    pygame.draw.line(screen, "white", pointsThree[0], pointsThree[2])
    pygame.draw.line(screen, "white", pointsThree[0], pointsThree[1])
    pygame.draw.line(screen, "white", pointsThree[2], pointsThree[3])
    pygame.draw.line(screen, "white", pointsThree[1], pointsThree[3])
    return

## Draw cell two left
def __drawTwoLeft():
    pygame.draw.line(screen, "white", pointsThree[0], (pointsTwo[0][0], pointsThree[0][1]))
    pygame.draw.line(screen, "white", pointsThree[2], (pointsTwo[1][0], pointsThree[1][1]))
    return

## Draw cell two right
def __drawTwoRight():
    pygame.draw.line(screen, "white", pointsThree[2], (pointsTwo[2][0], pointsThree[2][1]))
    pygame.draw.line(screen, "white", pointsThree[3], (pointsTwo[3][0], pointsThree[3][1]))
    return

def drawThree():
    for i in range(4):
        pygame.draw.line(screen, "grey", pointsThree[i], pointsfour[i])
    return

## Draw the first-person view of the current Player location and directional orientation
## based on a submitted 4x3 local map

def drawView(localmap):
    global screen
    screen.fill('black')
    __drawZero()
    if localmap[3][0]:
        __drawZeroLeft()
    if localmap[3][2]:
        __drawZeroRight()
    if localmap[2][1]:
        __drawOne()
        if localmap[2][0]:
            __drawOneLeft()
        if localmap[2][2]:
            __drawOneRight()
        if localmap[1][1]:
            __drawTwo()
            if localmap[1][0]:
                __drawTwoLeft()
            if localmap[1][2]:
                __drawTwoRight()
            if localmap[0][1]:
                drawThree()
    pygame.display.flip()
    
    

In [95]:
## Main launch method

def main():
    global currentMap
    
    # Initialize the pygame window
    initialize()
    
    # Load up the first map
    currentMap = loadMap("MovementTestLevel.lvlmap")

    # Draw a first-person view of the initial map state
    drawView(getVisible())

    # Begin the event-capture game loop
    gameLoop()
    # print("All good")
    return

In [96]:
## Program launch code
if __name__ == '__main__':
    main()

(2, 4)
1
[1, 0, 1, 0, 1, 1, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 1, 1, 1]
[1, 0, 1, 0, 3, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 1, 1, 1, 1]
[1, 0, 1, 0, 0, 0, 0, 0, 1]
[1, 1, 1, 0, 1, 0, 0, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 1, 0]
[0, 0, 1, 0, 1, 0, 0, 1, 0]
[0, 0, 1, 1, 1, 0, 1, 1, 0]
(2, 4)
[0, 0, 0]
[0, 1, 1]
[0, 1, 0]
[0, 3, 0]
(1, 4)
[0, 0, 0]
[0, 0, 0]
[0, 1, 1]
[0, 3, 0]
K_UP pressed
(0, 4)
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 3, 1]
K_UP pressed
2
(0, 4)
[0, 0, 1]
[0, 1, 1]
[0, 1, 0]
[0, 3, 3]
K_RIGHT pressed
3
(0, 4)
[1, 1, 0]
[0, 3, 0]
[0, 3, 0]
[1, 3, 0]
K_RIGHT pressed
(1, 4)
[0, 0, 0]
[1, 1, 0]
[0, 3, 0]
[0, 3, 0]
K_UP pressed


SystemExit: 0

c:\Users\nmsmi\OneDrive\Documents\Coding projects\WizardryEngine\env\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
